# Projeto 3: Classificação binária brest cancer com tuning dos parâmetros

## Etapa 1: Importação das bibliotecas

In [ ]:
!pip install skorch

     |████████████████████████████████| 133kB 11.7MB/s 


In [ ]:
import pandas as pd
import numpy as np
import torch.nn as nn
from skorch import NeuralNetBinaryClassifier
import torch
import torch.nn.functional as F
from sklearn.model_selection import GridSearchCV
torch.__version__

'1.6.0+cu101'

## Etapa 2: Base de dados

In [ ]:
np.random.seed(123)
torch.manual_seed(123)

In [ ]:
previsores = pd.read_csv('entradas_breast.csv')
classe = pd.read_csv('saidas_breast.csv')

In [ ]:
previsores = np.array(previsores, dtype = 'float32')
classe = np.array(classe, dtype = 'float32').squeeze(1)

In [ ]:
previsores.shape

(569, 30)

In [ ]:
classe.shape

(569,)

## Etapa 3: Classe para estrutura da rede neural

In [ ]:
class classificador_torch(nn.Module):
  def __init__(self, activation, neurons, initializer):
    super().__init__()
    # 30 -> 16 -> 16 -> 1
    self.dense0 = nn.Linear(30, neurons)
    initializer(self.dense0.weight)
    self.activation0 = activation
    self.dense1 = nn.Linear(neurons, neurons)
    initializer(self.dense1.weight)
    self.activation1 = activation
    self.dense2 = nn.Linear(neurons, 1)
    initializer(self.dense2.weight)
    self.output = nn.Sigmoid()

  def forward(self, X):
    X = self.dense0(X)
    X = self.activation0(X)
    X = self.dense1(X)
    X = self.activation1(X)
    X = self.dense2(X)
    X = self.output(X)
    return X

## Etapa 4: Skorch

In [ ]:
classificador_sklearn = NeuralNetBinaryClassifier(module=classificador_torch,
                                                  lr = 0.001,
                                                  optimizer__weight_decay = 0.0001,
                                                  train_split=False)

## Etapa 5: Tuning dos parâmetros

In [ ]:
params = {'batch_size': [10],
          'max_epochs': [100],
          'optimizer': [torch.optim.Adam, torch.optim.SGD],
          'criterion': [torch.nn.BCELoss], #, torch.nn.HingeEmbeddingLoss],
          'module__activation': [F.relu, F.tanh],
          'module__neurons': [8, 16], 
          'module__initializer': [torch.nn.init.uniform_]} # _, torch.nn.init.normal_]}

In [ ]:
params

{'batch_size': [10],
 'criterion': [torch.nn.modules.loss.BCELoss],
 'max_epochs': [100],
 'module__activation': [<function torch.nn.functional.relu>,
  <function torch.nn.functional.tanh>],
 'module__initializer': [<function torch.nn.init.uniform_>],
 'module__neurons': [8, 16],
 'optimizer': [torch.optim.adam.Adam, torch.optim.sgd.SGD]}

In [ ]:
grid_search = GridSearchCV(estimator=classificador_sklearn, param_grid=params,
                           scoring = 'accuracy', cv = 2)
grid_search = grid_search.fit(previsores, classe)

  epoch    train_loss     dur
-------  ------------  ------
      1       37.3239  0.0362
      2       37.3239  0.0384
      3       37.3239  0.0368
      4       37.3239  0.0337
      5       37.3239  0.0345
      6       37.3239  0.0334
      7       37.3239  0.0341
      8       37.3239  0.0397
      9       37.3239  0.0354
     10       37.3239  0.0382
     11       37.3239  0.0434
     12       37.3239  0.0354
     13       37.3239  0.0395
     14       37.3239  0.0347
     15       37.3239  0.0355
     16       37.3239  0.0370
     17       37.3239  0.0347
     18       37.3239  0.0368
     19       37.3239  0.0360
     20       37.3239  0.0364
     21       37.3239  0.0365
     22       37.3239  0.0348
     23       37.3239  0.0349
     24       37.3239  0.0352
     25       37.3239  0.0349
     26       37.3239  0.0353
     27       37.3239  0.0356
     28       37.3239  0.0352
     29       37.3239  0.0345
     30       37.3239  0.0340
     31       37.3239  0.0354
     32   

IndexError: ignored

In [ ]:
melhores_parametros = grid_search.best_params_
melhor_precisao = grid_search.best_score_

In [ ]:
melhores_parametros

{'batch_size': 10,
 'criterion': torch.nn.modules.loss.BCELoss,
 'max_epochs': 100,
 'module__activation': <function torch.nn.functional.relu>,
 'module__initializer': <function torch.nn.init._make_deprecate.<locals>.deprecated_init>,
 'module__neurons': 8,
 'optimizer': torch.optim.adam.Adam}

In [ ]:
melhor_precisao

0.7524153694094391